In [1]:
import numpy as np
from datetime import datetime

# Load features previously extracted from training data

In [2]:
x_train_features = np.load('x_train_features.npy')
y_train = np.load('y_train.npy')

x_test_features = np.load('x_test_features.npy')
y_test = np.load('y_test.npy')

print(x_train_features.shape, y_train.shape, x_test_features.shape, y_test.shape)

(29404, 6272) (29404, 5) (4861, 6272) (4861, 5)


# Prepare new classification model

In [3]:
from pond.tensor import NativeTensor, PrivateEncodedTensor, PublicEncodedTensor
from pond.nn import Dense, Sigmoid, Reveal, Diff, Softmax, CrossEntropy, Sequential, DataLoader

In [4]:
classifier = Sequential([
    Dense(128, 6272),
    Sigmoid(),
    # Dropout(.5),
    Dense(5, 128),
    Reveal(),
    Softmax()
])

In [5]:
def accuracy(classifier, x, y, verbose=0, wrapper=NativeTensor):
    predicted_classes = classifier \
        .predict(DataLoader(x, wrapper), verbose=verbose).reveal() \
        .argmax(axis=1)
        
    correct_classes = NativeTensor(y) \
        .argmax(axis=1)
        
    matches = predicted_classes.unwrap() == correct_classes.unwrap()
    return sum(matches)/len(matches)

# [optional] Fine-tune using NativeTensor to get accuracy expectations

In [6]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    DataLoader(x_train_features, wrapper=NativeTensor), 
    DataLoader(y_train, wrapper=NativeTensor), 
    loss=CrossEntropy(), 
    epochs=3,
    learning_rate=.01,
    verbose=1
)
stop = datetime.now()

print("Elapsed:", stop - start)

2018-01-29 08:39:08.836028 Epoch 0
2018-01-29 08:39:29.404299 Epoch 1
2018-01-29 08:39:49.102325 Epoch 2
Elapsed: 0:01:01.768282


In [7]:
train_accuracy = accuracy(classifier, x_train_features, y_train)
test_accuracy  = accuracy(classifier, x_test_features,  y_test)

In [8]:
print("Train accuracy:", train_accuracy)
print("Test accuracy:",  test_accuracy)

Train accuracy: 0.92361583458
Test accuracy: 0.921826784612


# [optional] Fine-tune using PublicEncodedTensor to get baseline

In [9]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    DataLoader(x_train_features, wrapper=PublicEncodedTensor), 
    DataLoader(y_train, wrapper=PublicEncodedTensor),
    loss=CrossEntropy(), 
    epochs=3,
    learning_rate=.01,
    verbose=1
)
stop = datetime.now()

print("Elapsed:", stop - start)

2018-01-29 08:41:32.014263 Epoch 0
2018-01-29 12:18:44.271448 Epoch 1
2018-01-29 15:21:50.934498 Epoch 2
Elapsed: 9:22:20.037328


In [13]:
np.save('layer0_weights.npy', classifier.layers[0].weights.elements)
np.save('layer0_bias.npy',    classifier.layers[0].bias.elements)

np.save('layer2_weights.npy', classifier.layers[2].weights.elements)
np.save('layer2_bias.npy',    classifier.layers[2].bias.elements)

In [10]:
train_accuracy = accuracy(classifier, x_train_features, y_train)
test_accuracy  = accuracy(classifier, x_test_features,  y_test)

In [11]:
print("Train accuracy:", train_accuracy)
print("Test accuracy:",  test_accuracy)

Train accuracy: 0.923513807645
Test accuracy: 0.920798189673


# Fine-tune using PrivateEncodedTensor

In [ ]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    DataLoader(x_train_features, wrapper=PrivateEncodedTensor), 
    DataLoader(y_train, wrapper=PrivateEncodedTensor),
    loss=CrossEntropy(), 
    epochs=3,
    learning_rate=.01,
    verbose=1
)
stop = datetime.now()

print("Elapsed:", stop - start)

## Save encrypted weights for future prediction

In [17]:
np.save('layer0_weights_0.npy', classifier.layers[0].weights.shares0)
np.save('layer0_weights_1.npy', classifier.layers[0].weights.shares1)
np.save('layer0_bias_0.npy',    classifier.layers[0].bias.shares0)
np.save('layer0_bias_1.npy',    classifier.layers[0].bias.shares1)

np.save('layer2_weights_0.npy', classifier.layers[2].weights.shares0)
np.save('layer2_weights_1.npy', classifier.layers[2].weights.shares1)
np.save('layer2_bias_0.npy',    classifier.layers[2].bias.shares0)
np.save('layer2_bias_1.npy',    classifier.layers[2].bias.shares1)

In [18]:
train_accuracy = accuracy(classifier, x_train_features, y_train)
test_accuracy  = accuracy(classifier, x_test_features,  y_test)

In [20]:
print("Train accuracy:", train_accuracy)
print("Test accuracy:",  test_accuracy)

Train accuracy: 0.923751870494
Test accuracy: 0.921209627649
